# Goal: Visualize Data Sources for ReachMaster
### ReachMaster is a data-rich experimental system that combines multiple readouts of experimental, kinematic, and voltage variables. These variables describe what exactly occurs during an experiment in ReachMaster. For more information on these variables, please see the data tutorial.
## For our case, we are interested visualizing the time-series of the following variables
### Experimental data: Lick detection, trial length, moving (time-course, length)..,inRWZone, rob command pos, rob pos, 
### Kinematic data: x, y, z and p of bodyparts over time
## We will combine these two data sources to create visuals describing how data sources vary from trial to trial due to various behaviors. 
## We will then examine these time-series variables over entire experimental sessions between different rats, days, sessions, and dimensions. 
## Together, this will allow us to estimate approaches to classifying different trial types as well as estimating phases in behavior.


In [1]:
import numpy as np
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as axes3d
import seaborn as sns
import pandas
import scipy
import pdb
import pandas as pd
import pickle
from Analysis_Utils import import_robot_data,preprocessing_df
%matplotlib tk
from Analysis_Utils import query_df
import DataStream_Vis_Utils as utils
### MAKE MATLAB BEAUTIFUL
CB91_Blue = '#2CBDFE'
CB91_Green = '#47DBCD'
CB91_Pink = '#F3A0F2'
CB91_Purple = '#9D2EC5'
CB91_Violet = '#661D98'
CB91_Amber = '#F5B14C'
color_list = [CB91_Blue, CB91_Pink, CB91_Green, CB91_Amber,
              CB91_Purple, CB91_Violet]
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=color_list)
sns.set(font='Franklin Gothic Book',
        rc={
 'axes.axisbelow': False,
 'axes.edgecolor': 'lightgrey',
 'axes.facecolor': 'None',
 'axes.grid': False,
 'axes.labelcolor': 'dimgrey',
 'axes.spines.right': False,
 'axes.spines.top': False,
 'figure.facecolor': 'white',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': 'dimgrey',
 'xtick.bottom': False,
 'xtick.color': 'dimgrey',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'dimgrey',
 'ytick.direction': 'out',
 'ytick.left': False,
 'ytick.right': False})
sns.set_context("notebook", rc={"font.size":16,
                                "axes.titlesize":20,
                                "axes.labelsize":18})

plt.legend(frameon=False)




def preprocessing_df(df):
    list_of_names=['r','t1','t2','x_p','y_p','z_p','RW']
    #shaking = xform_variables_to_numpy(df,list_of_names)
    shaking = dim_change(df)
    shaking=date_wipe(shaking)
    return shaking
    


def xform_variables_to_numpy(df, list_of_names):
    for i in list_of_names:
        for k, row_value in df[i].iteritems():
            x=str(df[i][k])
            x=x.strip('[,]')
            x=x.replace('...',"").replace('\n',"").replace('[',"").replace(']',"").replace('r',"")
            x=x.replace('..',"").replace('xp',"").replace('time',"").replace('object',"")
            x=x.replace(',',"").replace('inRewardWin',"")
            x=x.replace('Name:',"").replace('_',"").replace('robmoving',"")
            x=x.replace('rob_moving',"").replace("Length","").replace(':',"").replace("dtype","").replace("int64","")
            x=x.replace('inRewardWin',"")
            x=str(x).split()
            df.loc[i,k] = [pd.Series(np.asarray(x, dtype=np.float32)).reset_index()]
    return df


def date_wipe(df):
    d=[]
    for index, row_value in df['Date'].iteritems():
        if "_" in row_value:
            d.append((row_value)[4:6])
        else:
            d.append((row_value)[5:7])
    df['Date']=d
    return df


def dim_change(df):
    for k,x in enumerate(df['dim']):
        try:
            if "cone" in x:
                df['dim'][k] = 3
            elif "20mm" in x:
                df['dim'][k] = 0
            elif "10mm" in x:
                df['dim'][k] = 0
            else:
                df['dim'][k] = 0
        except:
            continue
    return df


No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [2]:
# load in robot data
import pickle
def import_robot_data(df_path):
    df=pd.read_pickle(df_path)
    df = preprocessing_df(df)
    return df

df = import_robot_data('/home/kallanved/Documents/default_save_new.pickle')
df = df.reset_index(drop=True)
# load in camera data
#kin_df = pickle.load( open( "k_data.pkl", "rb" ) ) # imports a list of kinematic variables for experiments from RM16
df.head()

<ipython-input-1-a48a2a63676f>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dim'][k] = 3


,rat,S,Date,dim,time,m_start,m_stop,SF,t_m,lick,...,RW,r_start,r_stop,r,t2,t1,exp_response,x_pot,y_pot,z_pot
0,RM9,S2,18,3,"[20.859866697184316, 20.8858667200725, 20.9028...","[31.668866712443105, 67.45086677729296, 105.33...","[35.674866754404775, 71.4538667982738, 109.495...","[2, 3, 4, 5, 6, 7, 8, 9, 13, 17, 19, 20, 21, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 35.6913, 35.6982, 36.564233333333334, 10...",...,0 0 1 0 2 0 3 0 4 ...,"[1513, 6563, 12020, 13242, 20341, 21483, 22472...","[2077, 7124, 12612, 13736, 21060, 22046, 23017...",916.8 916.8 655.4 916.8 614.4 614.4 655.4 916....,491.3 571.1 361.6 571.1 531.0 531.0 707.5 491....,648.8 648.8 794.8 648.8 608.0 608.0 794.8 648....,0 s 1 s 2 s 3 s 4 ...,"[9682, 9658, 9682, 9682, 9658, 9690, 9682, 954...","[6897, 6897, 6921, 6905, 6905, 6921, 6905, 690...","[6049, 6073, 6065, 6057, 6065, 6057, 6065, 605..."
1,RM9,S1,18,3,"[20.414766451136302, 20.440766474024485, 20.48...","[89.86176647783918, 99.30376662851972, 174.025...","[93.86576666475935, 103.30476655604048, 179.71...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 179.05716666666666, 179.09396666666666, ...",...,0 0 1 0 2 0 3 0 4 ...,"[9864, 11201, 21864, 24920, 26074, 27904, 2992...","[10430, 11766, 22667, 25452, 26535, 28404, 304...",614.4 614.4 614.4 614.4 614.4 614.4 614.4 614....,531.0 531.0 531.0 531.0 531.0 531.0 531.0 531....,608.0 608.0 608.0 608.0 608.0 608.0 608.0 608....,0 s 1 s 2 s 3 s 4 ...,"[6505, 6505, 6489, 6505, 6497, 6433, 6497, 649...","[6529, 6241, 6529, 6521, 6513, 6505, 6521, 651...","[5641, 5625, 5641, 5641, 5641, 5641, 5665, 564..."
2,RM9,S3,18,3,"[19.026683361943583, 19.089683564122538, 19.10...","[34.658683331426005, 117.32368353169761, 142.7...","[38.665683539327006, 121.32568338673912, 146.1...","[2, 3, 6, 9, 12, 14, 15, 16, 18, 20, 21, 23]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 122.51423333333334, 122.51763333333334, ...",...,0 0 1 0 2 0 3 0 4 ...,"[2191, 14018, 17703, 18863, 21270, 31546, 3251...","[2753, 14597, 18205, 19352, 21850, 32097, 3301...",655.4 916.8 916.8 916.8 916.8 655.4 655.4 916....,707.5 571.1 571.1 571.1 491.3 361.6 707.5 491....,449.0 568.6 648.8 648.8 568.6 449.0 794.8 648....,0 s 1 s 2 s 3 s 4 ...,"[9754, 9754, 9754, 9746, 9754, 9746, 9738, 973...","[6001, 6001, 6009, 5993, 6009, 5977, 5961, 608...","[6057, 6073, 6057, 6065, 6057, 6081, 5985, 605..."
3,RM9,S4,18,3,"[21.447966702906342, 21.479966767756196, 21.52...","[34.8089667639415, 152.94096673342392, 159.957...","[38.812966712443085, 156.9469667753856, 163.96...","[4, 5, 7, 10, 11, 12, 13, 14, 15, 16, 19, 20, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 214.57526666666666, 214.68453333333332, ...",...,0 0 1 0 2 0 3 0 4 ...,"[1873, 18701, 19694, 20832, 26812, 32196, 3331...","[2439, 19269, 20261, 21394, 27761, 32686, 3388...",916.8 614.4 655.4 655.4 655.4 916.8 916.8 655....,571.1 531.0 361.6 707.5 361.6 491.3 491.3 361....,648.8 608.0 449.0 794.8 449.0 648.8 648.8 449....,0 s 1 s 2 s 3 s 4 ...,"[6513, 6529, 6513, 6481, 6537, 6489, 6529, 651...","[6465, 6465, 6433, 6457, 6457, 6457, 6449, 646...","[5625, 5609, 5617, 5625, 5609, 5617, 5633, 567..."
4,RM9,S1,19,3,"[20.34046666857398, 20.412466695276862, 20.439...","[36.99546687838233, 93.67546670672095, 103.992...","[40.99946682688392, 97.68146674868262, 107.997...","[3, 5, 9, 12, 13, 16, 19, 20, 21, 22, 23, 27, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 108.04173333333334, 108.04273333333333, ...",...,0 0 1 0 2 0 3 ...,"[2332, 10392, 11884, 13810, 15792, 18687, 2008...","[2898, 10977, 12461, 14321, 16370, 19248, 2064...",655.4 916.8 655.4 655.4 916.8 655.4 916.8 655....,361.6 491.3 707.5 707.5 571.1 361.6 491.3 707....,794.8 648.8 794.8 449.0 568.6 449.0 568.6 449....,0 s 1 s 2 s 3 ...,"[9730, 9722, 9722, 9554, 9722, 9722, 9730, 964...","[6857, 6865, 6849, 6873, 6857, 6857, 6873, 685...","[5225, 5225, 5233, 5225, 5241, 5193

# I will examine 4 different days.
## RM16, 9/20, s3, dim=3D (9pt3dcone)
## RM16, 9/20, s4, 
## RM16, 9/25, s2/3
## RM16, 9/28, s2

In [3]:
# Obtain Single Session Data
dd_1 = utils.get_single_trial(df,'20','S3','RM16')
dd_1 = dd_1.reset_index(drop=True)

dd_2 = utils.get_single_trial(df,'20','S4','RM16')
dd_2 = dd_2.reset_index(drop=True)

dd_3 = utils.get_single_trial(df,'25','S3','RM16')
dd_3 = dd_3.reset_index(drop=True)

dd_4 = utils.get_single_trial(df,'27','S4','RM16')
dd_4 = dd_4.reset_index(drop=True)

# Diff Rat (14)
dd_1_RM14 = utils.get_single_trial(df,'20','S3','RM14')
dd_1_RM14 = dd_1_RM14.reset_index(drop=True)

# Diff Dim
dd_5 = utils.get_single_trial(df,'17','S2','RM16')
dd_5 = dd_5.reset_index(drop=True)


In [4]:
moving = dd_1['moving'].values
moving[0].shape
plt.plot(moving[0])
plt.show()

In [5]:
rz = dd_1['RW'].values
rz[0].shape
plt.plot(moving[0])
plt.show()

## OK, let's start with the first dataframe, for RM16, 9/20, session 3 (dd_1)
## We would like to obtain the successful single trial time-series for the experimental variables we are interested in from this experimental session-wide dataframe.

In [6]:
def clean_exp_row(row_df):
    x=str(row_df)
    x=x.strip('[,]').strip('(,)')
    x=x.replace('..',"").replace('Name:',"").replace('in_Reward_Win',"").replace(',',"").replace('Length:','')
    x=x.replace('\n',"").replace('[',"").replace(']',"").replace('array',"").replace('int64',"").replace(':',"").replace('...',"").replace('dtype',"")
    x=str(x).split()
    x=np.asarray(x,dtype=np.float)
    return x


def clean_e_row(row_df):
    x=str(row_df)
    filter( lambda x: x in '0123456789.', row_df )
    x=x.replace('..',"").replace('Name:',"").replace('in_Reward_Win',"")
    x=str(x).split()

    return x


def extract_trial_data_from_session(df, start, stop, features):
    ### Code intended to extract single trial data features from the experimental dataframe from an experimental block. 
        
    ###
    # clean in-coming data 
    # Allocate numpy array
    sz=len(features)
    lz=int(stop-start)
    a = {}
    # slice dataframe into arrays for time-series visualization
    for c,v in enumerate(features):
        if 'time' in v:
            a.update({v:np.ravel(df['time'])})
            continue
        z=0
        ix = df[v].values[0]
        if isinstance(ix, np.ndarray):  
            ix=np.asarray(ix)
        else:
            ix=clean_e_row(ix)
            #ix=np.asarray(ix,dtype=np.float)
        #lx=ix.shape[0]
        #pdb.set_trace()
        a.update({v:np.ravel(ix)})
    #dataframe = pd.DataFrame({a},columns=features)
    return a


    
trial_list = []
features = ['moving','RW','lick','t_m','x_p','y_p','z_p','time']
# extract the trial time-series lists
start=dd_1['r_start'].values
start=start[0]
stop=dd_1['r_stop'].values
stop=stop[0]
for c,v in enumerate(start):
    #print(v,stop[c])
    trial_list.append(extract_trial_data_from_session(dd_1,v,stop[c],features))
    
print('Number of Trials Extracted is' + str(len(stop)))
trial_list[0].keys()

Number of Trials Extracted is50


dict_keys(['moving', 'RW', 'lick', 't_m', 'x_p', 'y_p', 'z_p', 'time'])

## We now have access to all the experimental data from RM16, 9/20, S3 Block. Let's look at the time-series data of some successful trials vs unsuccessful trials. A successful trial is defined as a reach and grasp of the handle that ends in the reward zone (indicated by the RW variable). There are several flavors of a failed trial with the ReachMaster system. The first is a potential failure to retract the handle during the trial and not fulfilling the successful requirement of a experiment. However, a reach has occured and can/ should be observed. Additionally, there are reaches that completely fail to make contact with the handle and miss. These trial types and their corresponding reaches can be found through classification of simultaneously recorded 3-D kinematic data of FAILED trials. Finally, there are trials that are triggered by noise in the video trigger of the camera's experimental system which cause false trials to be observed.

# Successful Trial

In [7]:
# successful trials = 1,2,3,4,6,20,21,18 (among others, chose these at random!)
successful_trials=[1,2,3,4,6,20,18]
failed_trials=[0,5,14,13,12]
trial_num=len(stop)
sf=str(dd_1['SF']).split()
s=dd_1['SF']
# Plot histogram of data catagories for successful and failed trails
def plot_succ_fail(df,sf):
    ### takes in blocks experimental dataframe, list of successful trials in a block
    ###
    # initialize the subplots for succ and failure
    plt.tight_layout()
    fig, (ax1, ax) = plt.subplots(2,figsize=(20, 12))
    ax.set_xlabel('Time (Exposures) ms')
    ax.set_ylabel('Robot Moving (Success!) ?')
    ax.set_title('Binary Experimental Time-Series Features: Success')
    ax1.set_xlabel('Time (Exposures)')
    ax1.set_ylabel('Robot Moving (Fail!) ?')
    #ax1.legend()
    ax1.set_title('Robot Position Experimental Time-Series Features: Failure')
    for i in range(0,trial_num-40):
        st=start[i]
        stx=stop[i]
        trial=trial_list[i]
        #ax.plot(trial['moving'],marker='x',color='r',alpha=0.01,label='Moving?')
        if i in (sf[0]):
            # binary features
            
            ax.plot(trial['moving'][st:stx+500],marker='.',color='k',alpha=0.6,label='Moving (Successful Trial)')
            #ax.plot(trial['RW'][0].T[st:stx+500],marker='.',color='y',label='In Reward Zone (Successful Trials)') # this should be present in all trials
        else:
            # plot these on another plot somewhere else (?)
            ax1.plot(trial['moving'][st:stx],marker='_',color='k',alpha=0.8,label='Moving (Failed Trial)')
    #plt.legend()
    plt.show()
plot_succ_fail(dd_1,s)
dd_1['RW'][0].shape

findfont: Font family ['Franklin Gothic Book'] not found. Falling back to DejaVu Sans.


(95171,)

## There is a distinct epoch in the moving time-series of the robot embedded into the experiment. The reaching start and end times begin and end around the same time period and there is a resulting end of movement around the same time window between trials. 
### We can ask 'how long' the robot is moving by calculating the length of the movement vector the robot has. I won't do this right this second, but I will do this across multiple blocks
## Let's see where all the other experimental variables fit in

In [8]:
successful_trials=[1,2,3,4,6,20,18]
failed_trials=[0,5,14,13,12]
trial_num=len(stop)
sf=str(dd_1['SF']).split()
s=dd_1['SF']
def find_len_moving(moving,time):
    for c, v in moving:
        if v == 1:
            if moving[v-1] == 0:
                start=c
            if moving[v+1] == 0:
                stop=c
        else:
            continue
        moving_length=time[stop]-time[start]
    return moving_length
# Plot histogram of data catagories for successful and failed trails
def plot_succ_fail(df,sf):
    ### takes in blocks experimental dataframe, list of successful trials in a block
    ###
    # initialize the subplots for succ and failure
    plt.tight_layout()
    fig, (ax, ax1) = plt.subplots(2,figsize=(20, 12))
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Robot Moving (Success!) ?')
    ax.set_title('Binary Experimental Time-Series Features: Failure')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Robot Moving (Fail!) ?')
    #ax1.legend()
    ax1.set_title('Robot Position Experimental Time-Series Features: Success')
    for i in range(0,trial_num):
        st=start[i]
        stx=stop[i]
        trial=trial_list[i]
        time=trial['time'][0][st:stx+500]
        start_time= trial['time'][0][st:stx]
        exp_time=np.asarray(time,dtype=float)-time[0] # this is normalized trial time
        REW_time=start_time[-1]-start_time[0]
        #ax.plot(trial['moving'],marker='x',color='r',alpha=0.01,label='Moving?')
        if i in (sf[0]):
            ax1.plot(exp_time,trial['moving'][st:stx+500],marker='.',color='k',alpha=0.6,label='Moving (Successful Trial)')
            #ax1.vlines(REW_time,ymin=0.5, ymax=0.5)
            ax1.scatter(REW_time,1,marker='v',color='g',s=450,label='In Reward Zone (Successful Trials)') # this should be present in all trials
        else:
            # plot these on another plot somewhere else (?)
            ax.vlines(4.02,ymin=1,ymax=0,color='r',label='Exp_End')
            ax.plot(exp_time,trial['moving'][st:stx+500],marker='_',color='k',alpha=0.1,label='Moving (Failed Trial)')
    plt.show()
plot_succ_fail(dd_1,s)
dd_1['RW'][0].shape

findfont: Font family ['Franklin Gothic Book'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Franklin Gothic Book'] not found. Falling back to DejaVu Sans.


(95171,)

## As seen above, experiments begin to 
## Now let's look at another set of experimental features, robot movement, or interaction,  lengths. This feature is intended to let us know exactly how the rat is interacting with the handle (more, less, well behaved?). With kinematic tracking, we can seperate out well-behaved trials (release of handle as it moves to next position) vs poor trials (no release, re-grasp etc). 

## Single Trials

In [9]:
from matplotlib.lines import Line2D

# Re-Grasp maybe +2 seconds no lick
# 

def get_simple_lick_times(lick,start,stop, exp_time):
    lick_index=[]
    b=1
    #pdb.set_trace()
    
    for i in lick:
        #pdb.set_trace()
        if start <= i <= stop+1:
            lick_index.append(i-start) 
        # get lick start/stop times
    #pdb.set_trace()
    try:
        lick_index=np.asarray(lick_index,dtype=float)
    except:
        lick_index=[0,0]
    try:
        lick_index=lick_index-lick_index[0] #normalize exp times
    except:
        lick_index=[0,0]
        b=0
    try:
        lick_start=lick_index[0]
    except:
        lick_start=0
    try:
        lick_end=lick_index[-1]
    except:
        lick_end=0
    
    
    return lick_start,lick_end, lick_index,b


def rz_length(rz_array,time_array):
    ### Takes in matched length Reward Zone and Normalized Time arrays (m_times)
        
    ###
    mask=np.zeros(len(time_array))
    for ix,ct in enumerate(rz_array):
        if ct == 1:
            mask[ix]=1
    # get rough amt of time spent in reward zone w/ # of exposures
    # exposures are variable, but only to 4th float place so we are ok (ms)
    rz_len = np. count_nonzero(mask == 1)
    flip=0
    for ji,jn in enumerate(mask):
        try:
            if jn == 1:
                if mask[ji-1]==0:
                    if mask[ji+1]==1:
                        if mask[ji-20]==0:
                            flip+=1
        except:
            flip=flip
    return rz_len,flip
        

def trial_length(start,stop,sf):
    t_length = []
    reward_region = []
    for i,x  in enumerate(start):
        if i in sf:
            t_length.append(stop[i]-x)
            reward_region.append(stop[i]+200-x+50)
    return t_length,reward_region

def plot_succ_fail_single(df,s,f,s_id='RM16 9/20 S3',trial_plot=True,plotting=True,
                          verbose=True,hist=True,save_fig=True,single_trial=[26],failed_trial=[0], kin_data=False,prob=False):
    ### takes in blocks experimental dataframe, list of successful trials in a block
    ###
    # initialize the subplots for succ and failure
    plt.tight_layout()
    moving=df['moving'].to_numpy()[0]
    RW=df['RW'].to_numpy()[0]
    licks = df['lick'].to_numpy()[0][1::]
    block_starts=df['m_start'].to_numpy()[0]
    block_stops=df['m_stop'].to_numpy()[0]
    block_time = df['time'].to_numpy()[0]
    m_start=df['m_start'].to_numpy()[0]
    m_stop=df['m_stop'].to_numpy()[0]
    trial_num=len(m_start)
    SF=df['SF'].to_numpy()[0]
    trial_lengths,r_lengths = trial_length(m_start,m_stop,SF)
    TSF=df['SF'].to_numpy()[0]
    if trial_plot:
        fx,g=plt.subplots(1,figsize=(20,5))
        g.plot(block_time,moving,color='k')
        g.plot(block_time,RW,color='g')
        g.scatter(licks,np.ones(len(licks)),color='b')
        g.scatter(m_start,np.zeros(len(m_start)),color='r',s=100)
        g.scatter(m_start,np.ones(len(m_start)),color='r',s=100)

        g.set_title('Full Block: Experimental Variable Features for block  ' + s_id)
        #g.scatter(m_start,np.zeros((m_start.shape[0])),color='r')
        #g.scatter(m_stop,np.ones(m_stop.shape[0]),color='r')
        plt.show()
    ### Sub-Trial Single Trial(s)Plot
    if plotting:
        fig, (ax, ax1) = plt.subplots(2,figsize=(20, 12))
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Movement Features (Failed Trials) ')
        ax.set_title('Experimental Time-Series Data Features: Failure')
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel('Movement Features (Successful Trial)')
        #ax1.legend()
        ax1.set_title('Experimental Data Time-Series Features: Success')
        moving_length = []
        s_move=[]
        f_move=[]
        t_rew_time=[]
        t_lick = []
        flips = 0
        ### function to extract single trial data + plots
        for i in range(0,trial_num):
            #print('Trial Number ' + str(i))
            st=start[i]
            stx=stop[i]
            st_real=m_start[i]
            stx_real=m_stop[i]
            time=df['time'][0][st:stx+500]
            start_time= df['time'][0][st:stx]
            rz = df['RW'].to_numpy()[0][st:stx+500]
            r_len=len(rz[st:stx])
            lick=df['lick'].to_numpy()[0]
            # obtain licking indices
            lstart,lstop,li,b=get_simple_lick_times(lick,st_real,stx_real,time)
            exp_time=np.asarray(time,dtype=float)-time[0] # this is normalized trial time
            REW_time=[0,start_time[-1]-start_time[0]]
            moving_length.append(REW_time)
            rz_len,rz_c = rz_length(rz,exp_time)
            if rz_len < 5: # check that no spurious values get by
                t_rew_time.append(rz_len)
            if b:
                t_lick.append(li[-1])
            if i in s:
                    s_move.append(REW_time)
            if i in f:
                f_move.append(REW_time)
            flips += rz_c
            if single_trial:
                if i in single_trial:
                    ax1.plot(exp_time,df['moving'].to_numpy()[0][st:stx+500],marker='_',color='k',alpha=0.9,
                             label='Moving (Successful Trial)')
                    ax1.scatter(exp_time,rz+0.05,color='g',alpha=0.8,label='Robot in Reward Area')
                    if b:
                        plt.scatter(li,np.ones(len(li)),color='b',s=200,alpha=0.9,label='Rat Licking')
                    ax1.scatter(REW_time,[0,1],marker='.',color='r',s=1080,label='Start and End of Experiment in ReachMaster') 
                    # this should be present in all trials
                if i in failed_trial:
                    plt.hlines(y=1, xmin=li[0], xmax=li[-1], linewidth=2, color='b',label='Rat Licking')
                    ax.plot(exp_time,df['moving'].to_numpy()[0][st:stx+500],marker='_',color='k',alpha=0.8,label='Moving (Failed Trial)')
                    ax.scatter(REW_time,[0,1],marker='.',color='r',s=1080,label='End of Experiment')
        colors = ['black', 'red', 'green','blue']
        lines = [Line2D([0], [0], color=c, linewidth=3, linestyle='--') for c in colors]
        labels = ['Robot Moving?', 'Experiment start and finish', 'Handle in Reward Area ?', 'Licking ?']
        plt.legend(lines, labels)
        #### Set up Binary Y-AXIS ###
        a=ax.get_yticks().tolist()
        a[1]='NO'
        a[2] = ''
        a[3] = ''
        a[4] = ''
        a[5] =''
        a[6]='YES'
        ax.set_yticklabels(a)
        ax1.set_yticklabels(a)
        if save_fig:
            plt.savefig('Exp_47_R16_s3_20.png')
        
        #plt.title('Single Trial Data Stream')
        plt.show()
    if verbose:
        print('Showing Trial Number  ' + str(s))
        print('Total Trials in Block: ' + str(trial_num))
        print('Number of estimated reaches ' +  str(flips))
        print('Average Length of Licking (in seconds) ' + str(np.mean(t_lick)))
        print('Average Time Spent in The Reward Area ' + str(np.mean(t_rew_time)))
        print('Average Successful Trial Length (in seconds)  ' + str(np.mean(trial_lengths)) )
    if hist:
        fsg, (ax2,ax3) = plt.subplots(2,figsize=(6,12))
        ax2.hist([moving_length,s_move,f_move],bins=5,color=['r','g','b'],alpha=0.9,label=['Total Movement','Successful Trial','Failed Trial'])
        ax2.set_title('Robot Movement to Command Position Duration')
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('Trials/Block')
        ax3.hist([t_lick,t_rew_time,trial_lengths],bins=5,color=['r','g','b'],alpha=0.9,label=['Lick Times','Time in Rew Zone','Length of Trial'])
        ax2.legend(loc='upper right')
        ax3.set_title('Trial Lengths')
        ax3.set_xlabel('Time (s)')
        ax3.set_ylabel('Trials/Block')
        ax3.legend(loc='upper right')
        plt.show()
    return moving_length, s_move, f_move, trial_num, flips, t_lick, t_rew_time,trial_lengths

moving_length5, s_move5, f_move5,t,flips,tl,trw,tlen=plot_succ_fail_single(dd_1,dd_1['SF'].to_numpy()[0],[15])

Showing Trial Number  [ 1  2  3  4  6  7  8 10 17 18 19 20 21 22 23 24 25 26 27 28 30 31 32 33
 34 35 37 38 39 40 43 44 45 46 47 48]
Total Trials in Block: 50
Number of estimated reaches 69
Average Length of Licking (in seconds) 2.6877141666666695
Average Time Spent in The Reward Area 0.2
Average Successful Trial Length (in seconds)  4.218388888570997


# It is now obvious that different trial types can be distinguished by basic experimental features. Let's dive into single trial examples with these features. The first: The successful single reach

# The successful single reach is distinguished from other trial types through several feature differences. Plotting the single trial variables of robot movement, reward zone positioning, trial beginning and end, and licking as a time series for each of these trial types with accompanying kinematic data and raw video is the start of our behavioral classification.

In [10]:
moving_length5, s_move5, f_move5,t,flips,tl,trw,tlen=plot_succ_fail_single(dd_2,dd_2['SF'].to_numpy()[0],[0])

Showing Trial Number  [ 3  4  5  6  7  8  9 11 12 15 16 17 19 21 23 24 25 26 29 31 32 33 34 35
 36]
Total Trials in Block: 38
Number of estimated reaches 31
Average Length of Licking (in seconds) 2.3879031249999976
Average Time Spent in The Reward Area 0.037037037037037035
Average Successful Trial Length (in seconds)  4.001799993515014


In [11]:
moving_length3, s_move3, f_move5,t,flips,tl,trw,tlen=plot_succ_fail_single(dd_3,dd_3['SF'].to_numpy()[0],[0],s_id=
                                                                          'RM16 9/25 S3')

Showing Trial Number  [4 8 9]
Total Trials in Block: 11
Number of estimated reaches 0
Average Length of Licking (in seconds) 0.9000333333333117
Average Time Spent in The Reward Area 0.0
Average Successful Trial Length (in seconds)  3.755666732788086


In [12]:
moving_length3, s_move3, f_move5,t,flips,tl,trw,tlen=plot_succ_fail_single(dd_4,dd_4['SF'].to_numpy()[0],[0],s_id=
                                                                          'RM16 9/27 S4')

IndexError: list index out of range

## I am now going to add the raw video peice of the puzzle. The video is extracted from trials using code found at https://github.com/BouchardLab/ReachMaster/blob/development/software/HDF%20File%20Scripts/Automatic_Trial_Videos_Extraction%20(Robot%20Data-HDF%20file).ipynb developed by Harnoor Dhingra.
## This code splits each camera feed into trial time resolution (eg hundreds of frames vs hundreds of thousands), returning both the raw images and a resulting video from a given camera feed. We have currently 3.

## Results of the code are then used to display the video behavior at a given time-point in a trial, super-imposed over the trial data.

## We view the video data seperately, matching the timesteps of images to specific timesteps of the experiment, to verify and observe behavioral changes w.r.t. experimental features.

# Now, I'm going to add in two new peices for this figure. The first, robot movement, is represented as a time-series plot of both predicted movement (from our pipeline) and movement reconstructed from our robots inverse and forward kinematics and potentiometer readings. The process to find this is found in a different notebook, titled (handle matching). The second is a plot of the 3-D coordinates and velocities from a given bodypart during the trial and the RMSE from the reconstructed coordinates.

In [ ]:
def pkl_to_df(pickle_file):
    # unpickle file
    unpickled_list = pd.read_pickle(pickle_file)
    
    # true if have encountered a dataframe in list
    encountered_df = False
    
    # iterate through list
    for i in range(len(unpickled_list)):
        rat_df1 = unpickled_list[i]
        if ((rat_df1 is not 0) and (type(rat_df1) is not list)):
            
            if (not encountered_df):
                encountered_df = True

                # create a new dataframe (copy of original), then removes levels corresponding to (rat, date, session, dim)
                df1 = rat_df1.droplevel([0, 1, 2, 3], axis=1)

                # inserts columns and data for (rat, date, session, dim)
                pos_arr = [0, 1, 1, 3] # order to insert columns (rat, date, session, dim)
                for i in range(4):
                    col_name = rat_df1.columns.names[i]
                    val = rat_df1.columns.levels[i][0]
                    df1.insert(pos_arr[i], col_name, val)
                
            else:    
                df2 = rat_df1.droplevel([0,1,2,3], axis = 1)
                
                pos_arr = [0, 1, 1, 3] # order to insert columns (rat, date, session, dim)
                for i in range(4):
                    col_name = rat_df1.columns.names[i]
                    val = rat_df1.columns.levels[i][0]
                    df2.insert(pos_arr[i], col_name, val)
                df1 = pd.concat([df1, df2], axis=0, sort=False) # concat new df to existing df
                
    # sets index of new df
    df1 = df1.set_index(['rat', 'date', 'session', 'dim'])
    return df1



# Uncomment lines below to call function
d=pkl_to_df('/home/kallanved/Downloads/kin1_datanew.pkl')


In [ ]:
import matplotlib.pyplot as plt
%matplotlib tk

In [ ]:
##### from scipy.interpolate import make_interp_spline
from scipy.interpolate import CubicSpline

def interpolate_interval(xt1,yt1,zt1,kinematics=False): # this interpolates coords,cpu intensive
    frame_axis = range(0,xt1.shape[0],1)
    c = frame_axis
    cs_x = CubicSpline(frame_axis, xt1.squeeze())
    cs_y = CubicSpline(frame_axis, yt1.squeeze())
    cs_z = CubicSpline(frame_axis, zt1.squeeze())

    x1_s = cs_x(frame_axis)
    y1_s = cs_y(frame_axis)
    z1_s = cs_z(frame_axis)
    if kinematics:
        x1_vel = cs_x(frame_axis, 1)
        y1_vel = cs_y(frame_axis, 1)
        z1_vel = cs_z(frame_axis, 1)

        x1_acc = cs_x(frame_axis, 2)
        y1_acc = cs_y(frame_axis, 2)
        z1_acc = cs_z(frame_axis, 2)
    return x1_s,y1_s,z1_s


def get_pos_arrays_from_kin_df(df,part,start,stop):
    # we want x, y and z from whatever dataframe we get.
    xa=df[(part,'X')].to_numpy()
    xa=filter_obvious_outliers(xa,9)
    ya=df[(part,'Y')].to_numpy()
    ya=filter_obvious_outliers(ya,9)
    za=df[(part,'Z')].to_numpy()
    za=filter_obvious_outliers(za,9)
    return xa,ya,za


def get_prob_arrays_from_kin_df(df,part,start,stop):
    xp=np.absolute(df[(part,'prob_x')].to_numpy())
    yp=np.absolute(df[(part,'prob_y')].to_numpy())
    zp=np.absolute(df[(part,'prob_z')].to_numpy())
    return xp,yp,zp


def get_rmse_arrays_from_kin_df(df,part,start,stop):
    rmse=np.absolute(df[(part,'rmse')].to_numpy())
    return rmse


def get_session_kin_data(d,rat,date,session):
    kim_df=d[d.index.get_level_values('rat') == rat][d.index.get_level_values('date') == date]
    kc=kim_df[kim_df.index.get_level_values('session') == session]
    return kc


def iter_kin_array(kin_dfx,st,stx,nfilt=5):
    feat_name=[]
    x_arr=[]
    y_arr=[]
    z_arr=[]
    hx_arr=[]
    hy_arr=[]
    hz_arr=[]
    for (columnName, columnData) in kin_dfx.iteritems():
        feat_name.append(columnName[0])
        #pdb.set_trace()
        if 'B Handle' in columnName[0]:
            if columnName[1] == 'X':
                hx_arr.append(filter_obvious_outliers(columnData.values[st:stx],nfilt))
            if columnName[1] == 'Y':
                hy_arr.append(filter_obvious_outliers(columnData.values[st:stx],nfilt))
            if columnName[1] == 'Z':
                hz_arr.append(filter_obvious_outliers(columnData.values[st:stx],nfilt))
        if columnName[1] == 'X':
            x_arr.append(filter_obvious_outliers(columnData.values[st:stx],nfilt))
        if columnName[1] == 'Y':
            y_arr.append(filter_obvious_outliers(columnData.values[st:stx],nfilt))
        if columnName[1] == 'Z':
            z_arr.append(filter_obvious_outliers(columnData.values[st:stx],nfilt))
    handle_coords=np.array((hx_arr,hy_arr,hz_arr))
    return x_arr,y_arr,z_arr,feat_name,handle_coords


def get_kin_velocity(kin_df,st,stx,time):
    # integrated velocity: compute integrated velocity across all bodyparts, handle for each frame of a trial
    ix,iy,iz,fete,handles=iter_kin_array(kin_df,st,stx)
    int_velocity_x=np.diff(ix)/np.diff(time)
    int_velocity_y=np.diff(iy)/np.diff(time)
    int_velocity_z=np.diff(iz)/np.diff(time)
    #pdb.set_trace()
    # split into left and right arm vectors
    int_velocity_x_r = int_velocity_x[15:27,:]
    int_velocity_x_l = int_velocity_x[3:14,:]
    int_velocity_y_r = int_velocity_y[15:27,:]
    int_velocity_y_l = int_velocity_y[3:14,:]
    int_velocity_z_r = int_velocity_z[15:27,:]
    int_velocity_z_l = int_velocity_z[3:14,:]
    int_velocity_l=np.sqrt((int_velocity_x_l**2)+(int_velocity_y_l**2)+(int_velocity_z_l**2))# convert to cm for better visualization
    int_velocity_r=np.sqrt((int_velocity_x_r**2)+(int_velocity_y_r**2)+(int_velocity_z_r**2))
    # normalize velocities
    int_velocity_l = int_velocity_l / np.sqrt(np.sum(int_velocity_l**2))
    int_velocity_r = int_velocity_r / np.sqrt(np.sum(int_velocity_r**2))
    pos_array=np.array((ix,iy,iz))
    handle_velocity=np.abs(np.diff(handles)/np.diff(time))
    handle_velocity=handle_velocity/np.sqrt(np.sum(handle_velocity**2))
    return int_velocity_l,int_velocity_r, handle_velocity, pos_array,fete


def xform_coords(xp,yp,zp):
    x=[]
    y=[]
    z=[]
    r=[]
    theta=[]
    phi=[]
    for i,vv in enumerate(xp):    
        r1,theta1,phi1,x1,y1,z1 = utils.forward_xform_coords(xp[i],yp[i],zp[i])
        r.append(r1)
        theta.append(theta1)
        phi.append(phi1)
        x.append(x1)
        y.append(y1)
        z.append(z1)

    return x,y,z



def plot_succ_fail_single_kin(df,kin_df,s,f,s_id='RM16 9/20 S3',trial_plot=True,plotting=True,
                          verbose=True,hist=False,save_fig=False,single_trial_num=[23],failed_trial=[9], kin_data=False,prob=False):
    ### takes in blocks experimental dataframe, list of successful trials in a block
    ###
    # initialize the subplots for succ and failure
    plt.tight_layout()
    rob_plot=False
    kin_plot=False
    failed_trial=False
    kin_measurements=True
    single_trial=False
    moving=df['moving'].to_numpy()[0]
    RW=df['RW'].to_numpy()[0]
    licks = df['lick'].to_numpy()[0][1::]
    block_starts=df['m_start'].to_numpy()[0]
    block_stops=df['m_stop'].to_numpy()[0]
    block_time = df['time'].to_numpy()[0]
    m_start=df['m_start'].to_numpy()[0]
    m_stop=df['m_stop'].to_numpy()[0]
    trial_num=len(m_start)
    SF=df['SF'].to_numpy()[0]
    trial_lengths,r_lengths = trial_length(m_start,m_stop,SF)
    TSF=df['SF'].to_numpy()[0]
    
    ### Potentiometer Variables ###
    xp=dd_1['x_pot'].values[0]
    yp=dd_1['y_pot'].values[0]
    zp=dd_1['z_pot'].values[0]
    r1,theta1,phi1,x1,y1,z1 = utils.forward_xform_coords(xp[::214],yp[::214],zp[::214]) # downsample potentiometer readings to match camera exposures
    grad=np.gradient(np.asarray([x1,y1,z1]),axis=0)
    acc = np.gradient(grad)
    if trial_plot:
        fx,g=plt.subplots(1,figsize=(20,5))
        g.plot(block_time,moving,color='k')
        g.plot(block_time,RW,color='g')
        g.scatter(licks,np.ones(len(licks)),color='b')
        g.scatter(m_start,np.zeros(len(m_start)),color='r',s=100)
        g.scatter(m_start,np.ones(len(m_start)),color='r',s=100)
        
        g.set_title('Full Block: Experimental Variable Features for block  ' + str(s_id))
        #g.scatter(m_start,np.zeros((m_start.shape[0])),color='r')
        #g.scatter(m_stop,np.ones(m_stop.shape[0]),color='r')
        plt.show()
        
    if plotting:
        fig, (f1,fg,axel, ax1) = plt.subplots(4,figsize=(50, 12))
        #ax.set_xlabel('Time (s)')
        #ax.set_ylabel('Movement Features (Failed Trials) ')
        #ax.set_title('Experimental Time-Series Data Features: Failure')
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel('Movement Features (Successful Trial)'+ str(single_trial_num))
        #ax1.legend()
        ax1.set_title('Experimental Data Time-Series Features: Success')
        moving_length = []
        s_move=[]
        f_move=[]
        t_rew_time=[]
        t_lick = []
        flips = 0
        plot_loop=False
        ### function to extract single trial data + plots
        for i in range(0,trial_num):
            if plot_loop:
                scr = input("What Trial are you interested in today? Enter 9001 to exit")
                i=int(scr)
                if i == 9001:
                    break
                else:
                    continue
            st=start[i]
            stx=stop[i]
            st_real=m_start[i]
            stx_real=m_stop[i]
            time=df['time'][0][st:stx+500]
            start_time= df['time'][0][st:stx]
            rz = df['RW'].to_numpy()[0][st:stx+500]
            r_len=len(rz[st:stx])
            lick=df['lick'].to_numpy()[0]
            # obtain licking indices
            lstart,lstop,li,b=get_simple_lick_times(lick,st_real,stx_real,time)
            exp_time=np.asarray(time,dtype=float)-time[0] # this is normalized trial time
            REW_time=[0,start_time[-1]-start_time[0]]
            moving_length.append(REW_time)
            rz_len,rz_c = rz_length(rz,exp_time)
           
            if rz_len < 5: # check that no spurious values get by
                t_rew_time.append(rz_len)
            if b:
                t_lick.append(li[-1])
            if i in s:
                    s_move.append(REW_time)
            if i in f:
                f_move.append(REW_time)
            flips += rz_c
            interpolate=False
            #pdb.set_trace()    
            if i == single_trial_num[0]: #plot kinematics of successful trial
                kin_plot=True
                single_trial=True
            ## Kinematic Plotting
            
            if kin_plot:
                kx,ky,kz=get_pos_arrays_from_kin_df(kin_df,'B Handle',st,stx)
                kp1x,kp1y,kp1z=get_pos_arrays_from_kin_df(kin_df,'Palm 1',st,stx)
                kp2x,kp2y,kp2z=get_pos_arrays_from_kin_df(kin_df,'Palm 2',st,stx)
                pr_1,pr_2,pr_3=get_prob_arrays_from_kin_df(kin_df,'B Handle',st,stx)
                rmse=get_rmse_arrays_from_kin_df(kin_df,'Handle',st,stx)
                #f1.scatter(exp_time,np.asarray(kx)[st:stx+500]*100,color='r',s=5,label='X Handle')
                #f1.scatter(exp_time,np.asarray(ky)[st:stx+500]*100,color='g',s=5,label='Y Handle')
                #f1.scatter(exp_time,np.asarray(kz)[st:stx+500]*100,color='b',s=5,label='Z Handle')
                f1.scatter(exp_time,np.asarray(kp1x)[st:stx+500]*100,color='r',s=15,label='X Palm 1')
                f1.scatter(exp_time,np.asarray(kp1y)[st:stx+500]*100,color='g',s=15,label='Y Palm 1')
                f1.scatter(exp_time,np.asarray(kp1z)[st:stx+500]*100,color='b',s=15,label='Z Palm 1')
                fg.scatter(exp_time,np.asarray(pr_1)[st:stx+500]*100,color='k',s=20,marker='p',label='X Palm 2')
                fg.scatter(exp_time,np.asarray(pr_2)[st:stx+500]*100,color='m',s=20,marker='p',label='Y Palm 2')
                fg.scatter(exp_time,np.asarray(pr_3)[st:stx+500]*100,color='y',s=20,marker='p',label='Z Palm 2')
                #pdb.set_trace()
                #fg.scatter(exp_time,np.asarray(pr_1)[st:stx+500]*100,color='r',s=5,label='Probabilities from Camera 1,Handle')
                #fg.scatter(exp_time,np.asarray(pr_2)[st:stx+500]*100,color='g',s=5,label='Probabilities from Camera 2,Handle')
                #fg.scatter(exp_time,np.asarray(pr_3)[st:stx+500]*100,color='b',s=5,label='Probabilities from Camera 3,Handle')
                #fg.scatter(exp_time,np.asarray(rmse)[st:stx+500],color='k',label='RMSE for each frame for a given bodypart.')
                f1.set_title('Positions over time for the indicated trial, Left Palm')
                fg.set_title('Positions over time for the indicated trial, Right Palm')
                plt.legend()
                kin_plot=False
            
            if single_trial:
                ax1.plot(exp_time,df['moving'].to_numpy()[0][st:stx+500],marker='_',color='k',alpha=0.9,
                         label='Moving (Successful Trial)')
                ax1.scatter(exp_time,rz+0.05,color='g',alpha=0.8,label='Robot in Reward Area')
                if b:
                    plt.scatter(li,np.ones(len(li)),color='b',s=200,alpha=0.9,label='Rat Licking')
                ax1.scatter(REW_time,[0,1],marker='.',color='r',s=1080,label='Start and End of Experiment in ReachMaster') 
                single_trial=False
                if failed_trial:
                    plt.hlines(y=1, xmin=li[0], xmax=li[-1], linewidth=2, color='b',label='Rat Licking')
                    ax1.plot(exp_time,df['moving'].to_numpy()[0][st:stx+500],marker='_',color='k',alpha=0.8,label='Moving (Failed Trial)')
                    ax1.scatter(REW_time,[0,1],marker='.',color='r',s=1080,label='End of Experiment')
        if kin_measurements:

            int_velocity_l,int_velocity_r, handle_velocity_a, pos_arrays,fete_arr=get_kin_velocity(kin_df,st,stx,start_time)
            fixit1 = plt.figure(figsize=(20,6))
            axelz = fixit1.add_subplot(1,1,1,label='Kinematic Measurements')
            length_velocity_l = np.sum(int_velocity_l,axis=0)
            length_velocity_r = np.sum(int_velocity_r,axis=0)
            handle_velocity_a=np.sum(np.sum(handle_velocity_a,axis=0),axis=0)
            start_times=np.array(start_time)[0:length_velocity_l.shape[0]]
            axel.scatter(start_times,length_velocity_l,color='r',label='Posture Vector Normalized Velocity Left Arm (m)')
            axel.scatter(start_times,length_velocity_r,color='g',label='Posture Vector Normalized Velocity Right Arm (m)')
            axel.scatter(np.array(start_time)[0:handle_velocity_a.shape[0]],handle_velocity_a,color='b',label='Handle Normalized Velocity (m)')
            axel.set_xlabel('Time (s)')
            axel.set_ylabel('Normalized Velocity (m)')
            axel.legend()
        colors = ['black', 'red', 'green','blue']
        lines = [Line2D([0], [0], color=c, linewidth=3, linestyle='--') for c in colors]
        labels = ['Robot Moving?', 'Experiment start and finish', 'Handle in Reward Area ?', 'Licking ?']
        plt.legend(lines, labels)
        #### Set up Binary Y-AXIS ###
        a=ax1.get_yticks().tolist()
        a[1]='NO'
        a[2] = ''
        a[3] = ''
        a[4] = ''
        a[5] =''
        a[6]='YES'
        ax1.set_yticklabels(a)
        if save_fig:
            plt.savefig('Exp_47_R16_s3_20_placeholder.png')
        
        #plt.title('Single Trial Data Stream')
        plt.show()
        
    if verbose:
        print('Showing Trial Number  ' + str(s))
        print('Total Trials in Block: ' + str(trial_num))
        print('Number of estimated reaches ' +  str(flips))
        print('Average Length of Licking (in seconds) ' + str(np.mean(t_lick)))
        print('Average Time Spent in The Reward Area ' + str(np.mean(t_rew_time)))
        print('Average Successful Trial Length (in seconds)  ' + str(np.mean(trial_lengths)) )
    if hist:
        fsg, (ax2,ax3) = plt.subplots(2,figsize=(6,12))
        ax2.hist([moving_length,s_move,f_move],bins=5,color=['r','g','b'],alpha=0.9,label=['Total Movement','Successful Trial','Failed Trial'])
        ax2.set_title('Robot Movement to Command Position Duration')
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('Trials/Block')
        ax3.hist([t_lick,t_rew_time,trial_lengths],bins=5,color=['r','g','b'],alpha=0.9,label=['Lick Times','Time in Rew Zone','Length of Trial'])
        ax2.legend(loc='upper right')
        ax3.set_title('Trial Lengths')
        ax3.set_xlabel('Time (s)')
        ax3.set_ylabel('Trials/Block')
        ax3.legend(loc='upper right')
        plt.show()
    
        #axel1.scatter(exp_time,pos_arrays)
        #int_velocity, handle_velocity_a, pos_arrays,fete_arr
    return moving_length, s_move, f_move, trial_num, flips, t_lick, t_rew_time,trial_lengths



def filter_obvious_outliers(x,k):
    # Code for median filtering function taken from https://gist.github.com/bhawkins/3535131.
    assert k % 2 == 1, "Median filter length must be odd."
    assert x.ndim == 1, "Input must be one-dimensional."
    k2 = (k - 1) // 2
    y = np.zeros ((len (x), k), dtype=x.dtype)
    y[:,k2] = x
    for i in range (k2):
        j = k2 - i
        y[j:,i] = x[:-j]
        y[:j,i] = x[0]
        y[:-j,-(i+1)] = x[j:]
        y[-j:,-(i+1)] = x[-1]
    return np.median(y, axis=1)
rat='RM16'
date='0190920'
session='S3'
dd=get_session_kin_data(d,rat,date,session)
moving_length5, s_move5, f_move5,t,flips,tl,trw,tlen=plot_succ_fail_single_kin(dd_1,dd,dd_1['SF'].to_numpy()[0],[3])
# independent notebooks
# timecourses on 1 block
# easier to read
# quant, hist seperate
# single points, examine change

In [ ]:
scr = input("What Trial are you interested in today? Enter 9001 to exit")
                single_trial_num=int(scr)
                if single_trial_num == 9001:
                    break
                else:
                    single_trial=True
                single_trial=[single_trial]

In [ ]:
def filter_obvious_outliers(x,k):
    # Code for median filtering function taken from https://gist.github.com/bhawkins/3535131.
    assert k % 2 == 1, "Median filter length must be odd."
    assert x.ndim == 1, "Input must be one-dimensional."
    k2 = (k - 1) // 2
    y = np.zeros ((len (x), k), dtype=x.dtype)
    y[:,k2] = x
    for i in range (k2):
        j = k2 - i
        y[j:,i] = x[:-j]
        y[:j,i] = x[0]
        y[:-j,-(i+1)] = x[j:]
        y[-j:,-(i+1)] = x[-1]
    return np.median(y, axis=1)



def block_pos_extract(kin_df,et,el):
    x_arr = []
    y_arr = []
    z_arr = []
    feat_name=[]
    for (columnName, columnData) in kin_df.iteritems():
        feat_name.append(columnName[0])
        try:
            if columnName[1] == 'X':
                x_arr.append(filter_obvious_outliers(columnData.values,7))
            if columnName[1] == 'Y':
                y_arr.append(filter_obvious_outliers(columnData.values,7))
            if columnName[1] == 'Z':
                z_arr.append(filter_obvious_outliers(columnData.values,7))
        except:
            print('No filtering..')
            pdb.set_trace()
    # transform the coordinates in each block
    try:
        block=xform_array([x_arr,y_arr,z_arr],et,el)
    except:
        print('o')
    return block,feat_name




def reshape_posture_trials(a):
    pdb.set_trace()
    for ix,array in enumerate(a):
        yarr=ykin[ix]
        zarr=zkin[ix]
        if ix==0:
            arx=a

        else:
            ar=np.vstack((a))
            pdb.set_trace()
            arx= np.concatenate((arx,ar))
    return arx


def split_trial(posture_array,start,stop,window_length,pre):
    # posture array is timexfeatxcoords
    trials_list = []
    for i, jx in enumerate(start[0]):
        k=stop[0][i]
        trials_list.append(posture_array[jx-pre:jx+window_length,:,:])
    return trials_list



def onehot(r_df):
    try:
        m=len(r_df['r_start'].values[0])
    except:
        print('onehot vector error')
        #pdb.set_trace()
    sf=r_df['SF'].values[0]
    hot_vec = np.zeros(m, dtype=int)
    hot_vec[sf]=1
    return np.asarray(hot_vec)



def unwind_list(strial_list):
    for ix, trial in enumerate(strial_list):
        if ix == 0:
            tsx=trial
        if ix!=0:
            try:
                tsx=np.concatenate((tsx,trial),axis=0) 
            except:
                print('concat error in unwind_list')
                #pdb.set_trace()
    return tsx



def reshape_list(strial_list):
    #pdb.set_trace()
    try:
        length,width,height=np.array(strial_list[0]).shape
        num_trials = len(strial_list)
        # make empty array
        emp=np.empty((num_trials,length,width,height))
        # loop over len num_trials
        for iterz,trials in enumerate(strial_list):
            ts=np.array(trials)
            try:
                emp[iterz,:,:,:]=ts
            except:
                emp[iterz,:,:,:]=np.zeros((length,width,height))
    except:
        #pdb.set_trace()
        print('Problem with reshaping the list..')
    return emp


def make_s_f_trial_arrays_from_block(kin_array,robot_array,et,el,rat,date,kdate,session,window_length=900,pre=50):
    try:
        kim_df=kin_array[kin_array.index.get_level_values('rat') == rat][kin_array.index.get_level_values('date') == kdate]
        kc=kim_df[kin_array.index.get_level_values('session') == session]
    except:
        print('Not in kinematic array : Trial '+ rat + date + session)
        return 0,0,0
    r_df=utils.get_single_trial(robot_array,date,session,rat)
    a,feat_labels=block_pos_extract(kin_array,et,el)
    #arx=reshape_posture_trials(a)
    
    start=r_df['r_start'].values
    stop=r_df['r_stop'].values
    try:
        m=len(r_df['r_start'].values[0])
    except:
        print('Cant get indices')
        #pdb.set_trace()
        print(rat,date,kdate,session)
        return 0,0,np.zeros((81)).reshape(81,1)
    tt1 = split_trial(np.asarray(a),start,stop,window_length,pre)
    hot_vector= onehot(r_df)
    try:
        tt = unwind_list(tt1)
        #pdb.set_trace()
        tsz=reshape_list(tt1)
        #tt=tt.reshape(tt.shape[0],tt.shape[1]*tt.shape[2])
    except:
        #pdb.set_trace()
        print(rat,date,session + ' Is not in shape!')
    return tt, hot_vector, tt1, feat_labels,tsz


def xform_array(k_m,eit,cl):
    k=np.asarray(k_m)
    return np.dot(k.reshape(k.shape[2],27,3),eit)+cl
et=np.array([[ 0.82558746, -0.23079685, -0.51491569],
         [-0.10478134,  0.83395256, -0.54179701],
         [ 0.55446029,  0.50125437,  0.66431757]])
el=np.array([0.16738222, 0.18527773, 0.38104647])




trial_list,hot_vector,tt,feats,tsz = make_s_f_trial_arrays_from_block(dd,df,et,el,'RM16','20','0190920','S3')





In [ ]:
tsz.shape

In [ ]:
feat_labels = ['Robot Handle', 'Robot Handle Back','Nose','Shoulder 1', ' Forearm 1', 'Wrist 1', 'Palm 1','Index Finger Base 1', 'Index Finger Tip 1'
              'Middle Finger Base 1','Middle Finger Tip 1','Third Finger Base 1','Third Finger Tip 1','Fourth Finger Base 1','Fourth Finger Tip 1',
              'Shoulder 2', ' Forearm 2', 'Wrist 2', 'Palm 2','Index Finger Base 2', 'Index Finger Tip 2'
              'Middle Finger Base 2','Middle Finger Tip 2','Third Finger Base 2','Third Finger Tip 2','Fourth Finger Base 2','Fourth Finger Tip 2']
robot_feat_labels = ['Rew_Zone','Moving','XPot','YPot','ZPot','XPos','YPos','ZPos']

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.pyplot import cm
from celluloid import Camera

def system(s,color, ax,name):
    #pdb.set_trace()
    for k in range(0,10):
        ax.scatter(s[k,0],s[k,1],s[k,2],color=color[k],label=name[k])
    ax.legend()
    #plt.show()
    #camera.snap()
    
        
        
fig3d=plt.figure()
camera = Camera(fig3d)
ax=Axes3D(fig3d)
win=100
tnum=4
color=cm.rainbow(np.linspace(0,1,27))
for ilx in range(0,tt[1].shape[0]):
    entry=tt[tnum]
    system(entry[ilx,:,:].reshape(27,3),color, ax,feat_labels)
    
ax.legend()    
camera.snap()
animation = camera.animate()


In [ ]:
lick_data=dd_1['lick'][0]
lick_data
# inside of a trial, ask if licking occurs, make sure it lines up with the s/F of our trials
# if licking, compute length, get start and stop time indixes of the lick per trial, make histogram of licking times 
def is_lick_in_trial(lick_data,start,stop):
    #### start and stop are the 'm_start' and 'm_stop' vectors from the experimental dataframe.
    # These are in experimental computer time
    l_succ_start = []
    l_start = []
    l_stop=[]
    l_length=[]
    l_time=[]
    for i,t in enumerate(start):
        f=stop[i]
        # if lick_data is between the start and stop of experiment, let's pull the trial #, time vector for plotting, and the duration #
        # is l within?
        for c,l in enumerate(lick_data):
            if 0 < c <= len(lick_data) -1:
                if i <= l <= t:
                    l_succ_start.append(i) # append succ trial #
                    l_time.append(l)
                    if lick_data[c-1] < l - 5: # ask if past lick has been uninitiated for more than 5 seconds 
                        l_start.append(l)
                    if lick_data[c+1] > l+5: # is next lick entry more than 5 away?
                        l_stop.append(l)
    for s,x in enumerate(l_stop):
        strt=l_start[s]
        l_length.append(x-strt)
    l_start=np.unique(l_start)
    l_stop=np.unique(l_stop)
    l_succ_start=np.unique(l_succ_start)
    l_time=np.unique(l_time)
    return l_succ_start,l_start,l_stop,l_length,l_time



        

In [ ]:
lick=dd_1['lick'][0]
time=dd_1['time'][0]

l_succ_start,l_start,l_stop,l_length,l_time, lick_mask,exp_time, REW_time, moving_length, e_time = single_trial_lick(lick,time,start,stop,trial_list)

In [ ]:
ax=plt.figure(1,figsize=(20,10))
plt.plot(lick_mask)

plt.title('Licking across Block 3, 9/20 RM16')
plt.xlabel('time(s)')
plt.ylabel('licking (y/n)')
plt.show()

In [ ]:
suc,lstart,lstop,l_length,ltime=is_lick_in_trial(lick_data,dd_1['m_start'][0],dd_1['m_stop'][0])
ltime 